# M - Automated Essay Scoring
_School of Information Technology_<br>
_Monash University Malaysia_<br>
(c) Copyright 2020, Ian Tan & Jun Qing Lim

Steps

- Read dataset (ASAP)
- Extract features (into file) using EASE
- Conduct machine learning (Sci-kit Learn libraries)
    - Naive Bayes
    - SVR
    - BLRR (later)
- Evaluate (QWK)

## Import Libraries

In [67]:
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm #SVR is in SVM
from sklearn.metrics import accuracy_score, confusion_matrix

### Import the EASE functions, which is located in the ease folder.

In [68]:
import sys
sys.path.insert(1, 'ease')
import create
import grade 
import model_creator 
import predictor_extractor 
import predictor_set 
import util_functions
import essay_set
import feature_extractor

from essay_set import EssaySet
from feature_extractor import FeatureExtractor

## Read Dataset

AES (Hewlett Foundation dataset from Kaggle) in the folder `asap-aes`.  For this, we use the `training_set_rel3` for training and testing.  Note that the `test_set` and the `valid_set` cannot be used as they don't contain the scores and are meant for the competition to score the entries.

In [69]:
data_set = pd.read_csv("asap-aes/training_set_rel3.tsv", sep='\t', encoding="latin-1")

In [70]:
data_set['essay'] = [entry.lower() for entry in data_set['essay']] # lower case for all words in essay

There are 8 different essay sets.  As an overview:
- Sets 1 & 2 are of persuasive/narrative in the form of letters
- Sets 3, 4, 5 & 6 are source dependent response to a given essay
- Sets 7 & 8 are of persuasive/narrative in the form of story writing essays

These format makes it good for transfer learning.

In [71]:
data_set_1 = data_set[data_set['essay_set'] == 1]
data_set_2 = data_set[data_set['essay_set'] == 2]
#data_set_3 = data_set[data_set['essay_set'] == 3]
#data_set_4 = data_set[data_set['essay_set'] == 4]
#data_set_5 = data_set[data_set['essay_set'] == 5]
#data_set_6 = data_set[data_set['essay_set'] == 6]
data_set_7 = data_set[data_set['essay_set'] == 7]
data_set_8 = data_set[data_set['essay_set'] == 8]

As each set will retain the original index, we want each of them to have their own indexing so that it is easier to match the essay and the scores.

In [72]:
data_set_1 = data_set_1.reset_index() # resets index
data_set_2 = data_set_2.reset_index()
#data_set_3 = data_set_3.reset_index()
#data_set_4 = data_set_4.reset_index()
#data_set_5 = data_set_5.reset_index()
#data_set_6 = data_set_6.reset_index()
data_set_7 = data_set_7.reset_index()
data_set_8 = data_set_8.reset_index()

We use just the `essay` content and the respective `scores`.

In [73]:
# If you want for the whole dataset.
# Commented out as we will work on individual datasets
#essays = data_set['essay']
#scores = data_set['domain1_score']

In [74]:
essays_1 = data_set_1['essay']
scores_1 = data_set_1['domain1_score']
essays_2 = data_set_2['essay']
scores_2 = data_set_2['domain1_score']
#essays_3 = data_set_3['essay']
#scores_3 = data_set_3['domain1_score']
#essays_4 = data_set_4['essay']
#scores_4 = data_set_4['domain1_score']
#essays_5 = data_set_5['essay']
#scores_5 = data_set_5['domain1_score']
#essays_6 = data_set_6['essay']
#scores_6 = data_set_6['domain1_score']
essays_7 = data_set_7['essay']
scores_7 = data_set_7['domain1_score']
essays_8 = data_set_8['essay']
scores_8 = data_set_8['domain1_score']

Rename the `domain1_score` column to `score`.

In [75]:
scores_1.columns = "score"
scores_2.columns = "score"
#scores_3.columns = "score"
#scores_4.columns = "score"
#scores_5.columns = "score"
#scores_6.columns = "score"
scores_7.columns = "score"
scores_8.columns = "score"

THE ABOVE NEEDS TO BE PUT INTO A LOOP BUT I LEFT IT AS IS BECAUSE YOU CAN PICK AND CHOOSE EASILY INSTEAD.

### Create the essay sets

Again, these can be looped but I kept them separated for ease of readability and commenting out those that we don't need.  Each set takes a long time to process, and hence please be patient with this part.

In [76]:
e_set_1 = EssaySet()
e_set_2 = EssaySet()
#e_set_3 = EssaySet()
#e_set_4 = EssaySet()
#e_set_5 = EssaySet()
#e_set_6 = EssaySet()
e_set_7 = EssaySet()
e_set_8 = EssaySet()

In [77]:
for i in range(len(essays_1)):
    e_set_1.add_essay(essays_1[i], scores_1[i])

In [78]:
for i in range(len(essays_2)):
    e_set_2.add_essay(essays_2[i], scores_2[i])

Left out for sets 3 - 6 for now.

In [79]:
for i in range(len(essays_7)):
    e_set_7.add_essay(essays_7[i], scores_7[i])

In [80]:
for i in range(len(essays_8)):
    e_set_8.add_essay(essays_8[i], scores_8[i])

## Extract Features

Currently only doing for Set 1

In [81]:
f_extractor = FeatureExtractor()

In [82]:
length = f_extractor.gen_length_feats(e_set_1)
length_df_1 = pd.DataFrame(
    length, 
    columns = [
        'chars', 
        'words', 
        'commas', 
        'apostrophes', 
        'punctuations', 
        'avg_word_length',
        # new stuff
        'sentences',
        'questions',
        'avg_word_sentence',
        'POS', 
        'POS/total_words'
    ]
)

_*Exclude the prompts for the time being*_

In [83]:
# Merge this with the score based on the index
# We use the shallow features first
features = length_df_1
dataset = features.merge(scores_1, left_index=True, right_index=True)
dataset.columns = ['chars', 'words', 'commas', 'apostrophes', 'punctuations',
                   'avg_word_length', 'sentences', 'questions', 'avg_word_sentence',
                   'POS', 'POS/total_words', 'score']
X_1 = dataset.iloc[:,0:10].values.astype(float)
y_1 = dataset.iloc[:,11].values.astype(float)

Reshape the data and label

In [84]:
X_1.shape

(1783, 10)

In [85]:
y_1 = np.array(y_1).reshape(-1,1)
y_1.shape

(1783, 1)

In [86]:
### Split the train and test set
from sklearn.model_selection import train_test_split
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_1_test[:5, :])

[[ 7.]
 [ 8.]
 [10.]
 [ 7.]
 [10.]]


## Model Training

### Naive Bayes Training

In [87]:
model_1_nb = naive_bayes.MultinomialNB()
model_1_nb.fit(X_1_train, y_1_train.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

At this stage, the Naive Bayes model is called `model_nb_1`

### SVM Training

Use standard scaler for the data

In [109]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_1_trainSVM = sc_X.fit_transform(X_1_train)
y_1_trainSVM = sc_y.fit_transform(y_1_train)
X_1_testSVM = sc_X.transform(X_1_test)
y_1_testSVM = sc_y.transform(y_1_test)

In [110]:
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
model_svm_1 = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
model_svm_1.fit(X_1_trainSVM,y_1_trainSVM.ravel())

[LibSVM]

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=True)

In [111]:
y_1_trainSVM

array([[ 1.61636028],
       [-2.29361387],
       [ 0.31303557],
       ...,
       [ 0.31303557],
       [ 0.96469792],
       [-0.33862679]])

### BLRR (Later)

In [120]:
from sklearn.preprocessing import StandardScaler
sc_Xb = StandardScaler()
sc_yb = StandardScaler()
X_1_trainBLRR = sc_Xb.fit_transform(X_1_train)
y_1_trainBLRR = sc_yb.fit_transform(y_1_train)
X_1_testBLRR = sc_Xb.transform(X_1_test)
y_1_testBLRR = sc_yb.transform(y_1_test)

In [118]:
from sklearn import linear_model
model_1_BLRR = linear_model.BayesianRidge()
model_1_BLRR.fit(X_1_trainBLRR, y_1_trainBLRR.ravel())

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

## Prediction

We will be using the respective validation set and will have to also pre-process the data.

### Naive Bayes

In [89]:
y_1_predNB = model_1_nb.predict(X_1_test)

cm = confusion_matrix(y_1_test, y_1_predNB)
print(cm)

[[ 4  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0  0  0  0]
 [ 0  1  0  0  1  0  0  0  0  0  0]
 [ 0  0  3  2  7  4  4  0  0  0  0]
 [ 0  0  0  0  7  4 12  0  1  0  0]
 [ 0  0  0  1  4 10 93 16 19  1  0]
 [ 0  0  0  0  0  2 17 14 18  1  3]
 [ 0  0  0  0  1  1  7 23 34  6  6]
 [ 0  0  0  0  0  0  0  2  7  3  8]
 [ 0  0  0  0  0  0  0  0  2  1  5]]


### SVM

In [128]:
y_1_predSVM = model_svm_1.predict(X_1_testSVM)
y_1_predSVM = sc_y.inverse_transform(y_1_predSVM).round()

cm = confusion_matrix(y_1_test, y_1_predSVM)
print(cm)

[[ 0  1  3  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  4 10  6  0  0  0  0]
 [ 0  0  0  0  0 13  9  2  0  0  0]
 [ 0  0  0  0  3 13 85 36  7  0  0]
 [ 0  0  0  0  0  1 10 29 13  2  0]
 [ 0  0  0  0  0  0  3 28 39  7  1]
 [ 0  0  0  0  0  0  0  3 10  6  1]
 [ 0  0  0  0  0  0  0  0  3  5  0]]


In [130]:
y_1_predSVM

array([ 8.,  8.,  9.,  7.,  8.,  9., 11., 10.,  7., 10., 10.,  8.,  9.,
        9.,  9.,  9., 10.,  8.,  8.,  8.,  8.,  9.,  8., 11., 10., 10.,
        8.,  8., 10.,  9.,  8.,  9.,  9.,  8., 12.,  9.,  9., 11.,  8.,
        7.,  9.,  8.,  8.,  7., 11.,  8.,  9.,  8.,  7.,  8.,  9.,  9.,
        9.,  8.,  8., 10.,  9.,  8.,  8.,  8., 12.,  7.,  9.,  9., 10.,
        8.,  8.,  9.,  8., 10., 10.,  8., 10., 10.,  9., 10., 10.,  9.,
        8.,  9.,  7.,  8., 10.,  8., 11.,  9.,  9.,  8.,  9., 10., 11.,
       10., 10., 10.,  8.,  4., 10.,  8.,  8.,  9.,  7.,  8.,  7.,  9.,
        8.,  9., 10.,  7.,  9.,  9.,  8., 10.,  9., 11., 11.,  4.,  9.,
        9.,  6.,  8.,  8.,  9.,  9.,  9.,  9., 10.,  9.,  9.,  8.,  7.,
       10.,  9.,  7., 10.,  9.,  8.,  8.,  7.,  9.,  8.,  9.,  9.,  8.,
        8., 10.,  9.,  8.,  9.,  8.,  9., 11.,  9.,  7.,  9., 10.,  8.,
        9.,  7.,  8., 10.,  9.,  9.,  8.,  8.,  9.,  7.,  9.,  9.,  8.,
        9.,  7., 10., 10., 10.,  8.,  4., 10.,  9.,  7.,  9., 10

### BLRR (Later)

In [131]:
y_1_predBLRR = model_1_BLRR.predict(X_1_testBLRR)
y_1_predBLRR = sc_yb.inverse_transform(y_1_predBLRR).round()

cm = confusion_matrix(y_1_test, y_1_predBLRR)
print(cm)

[[ 0  0  4  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  6 14  0  0  0  0  0  0]
 [ 0  0  0  0 15  7  2  0  0  0  0]
 [ 0  0  0  3 25 74 38  4  0  0  0]
 [ 0  0  0  0  3 10 32  9  0  1  0]
 [ 0  0  0  0  0  5 26 36 10  1  0]
 [ 0  0  0  0  0  1  2  9  6  2  0]
 [ 0  0  0  0  0  0  0  3  3  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0]]


## Evaluation using QWK

QWK scores for NB, SVR and BLRR

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

### Naive Bayes

In [90]:
rpt = classification_report(y_1_test, y_1_predNB)
print(rpt)

              precision    recall  f1-score   support

         2.0       1.00      1.00      1.00         4
         3.0       0.00      0.00      0.00         0
         4.0       0.25      0.50      0.33         2
         5.0       0.00      0.00      0.00         2
         6.0       0.35      0.35      0.35        20
         7.0       0.19      0.17      0.18        24
         8.0       0.70      0.65      0.67       144
         9.0       0.25      0.25      0.25        55
        10.0       0.42      0.44      0.43        78
        11.0       0.25      0.15      0.19        20
        12.0       0.23      0.62      0.33         8

   micro avg       0.46      0.46      0.46       357
   macro avg       0.33      0.38      0.34       357
weighted avg       0.48      0.46      0.47       357



D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [92]:
print(cohen_kappa_score(y_1_test, y_1_predNB, weights="quadratic"))

0.7845553930784764


### SVM

In [113]:
rpt = classification_report(y_1_test, y_1_predSVM)
print(rpt)

              precision    recall  f1-score   support

         2.0       0.00      0.00      0.00         4
         3.0       0.00      0.00      0.00         0
         4.0       0.20      0.50      0.29         2
         5.0       0.00      0.00      0.00         2
         6.0       0.57      0.20      0.30        20
         7.0       0.35      0.54      0.43        24
         8.0       0.75      0.59      0.66       144
         9.0       0.29      0.53      0.38        55
        10.0       0.54      0.50      0.52        78
        11.0       0.30      0.30      0.30        20
        12.0       0.00      0.00      0.00         8

   micro avg       0.50      0.50      0.50       357
   macro avg       0.27      0.29      0.26       357
weighted avg       0.54      0.50      0.50       357



D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [114]:
print(cohen_kappa_score(y_1_test, y_1_predSVM, weights="quadratic"))

0.8006385343042965


### BLRR

In [132]:
rpt = classification_report(y_1_test, y_1_predBLRR)
print(rpt)

              precision    recall  f1-score   support

         2.0       0.00      0.00      0.00         4
         4.0       0.00      0.00      0.00         2
         5.0       0.00      0.00      0.00         2
         6.0       0.46      0.30      0.36        20
         7.0       0.26      0.62      0.37        24
         8.0       0.76      0.51      0.61       144
         9.0       0.32      0.58      0.41        55
        10.0       0.59      0.46      0.52        78
        11.0       0.32      0.30      0.31        20
        12.0       0.20      0.12      0.15         8
        13.0       0.00      0.00      0.00         0

   micro avg       0.48      0.48      0.48       357
   macro avg       0.26      0.26      0.25       357
weighted avg       0.55      0.48      0.49       357



D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [133]:
print(cohen_kappa_score(y_1_test, y_1_predBLRR, weights="quadratic"))

0.8005463638675472


# END

#### Collate the essay prompts
This consist of one essay from each set

In [ ]:
essay_prompts = []

# Takes a bit of time also :)
for i in range(1,9):
    file = "prompts/set" + str(i) + ".txt"
    f = open(file, "r", encoding="latin-1") # there are some 0x9x characters, hence need to specify encoding
    essay_prompts.append(f.read())
    
def get_essay_prompt(essay_set):
    return essay_prompts[essay_set-1]

In [ ]:
# Unsure how this works
e_set.update_prompt(get_essay_prompt(2))

# Need more explanation on how this works - look into EASE

prompts = f_extractor.gen_prompt_feats(e_set)
prompts_df = pd.DataFrame(prompts, columns = ['prompt_words', 'prompt_words/total_words', 'synonym_words', 'synonym_words/total_words'])

In [ ]:
e_set

In [ ]:
# Another process that takes sometime to process
unstemmed = util_functions.get_vocab_essays_count(e_set._text, e_set._score)
stemmed = util_functions.get_vocab_essays_count(e_set._clean_stem_text, e_set._score)

bow = list(map(lambda a,b:[a,b], unstemmed, stemmed))
bow_df = pd.DataFrame(bow, columns = ['unstemmed', 'stemmed'])

In [ ]:
features = pd.concat([length_df, prompts_df, bow_df], axis=1, sort=False)

In [ ]:
features.head()

In [ ]:
# Export features to a file for next stage (optional)
dataset = features.merge(scores, left_index=True, right_index=True)

In [ ]:
dataset.head()

In [ ]:
dataset.columns = ['chars', 'words', 'commas', 'apostrophes', 'punctuations',
       'avg_word_length', 'POS', 'POS/total_words', 'prompt_words',
       'prompt_words/total_words', 'synonym_words',
       'synonym_words/total_words', 'unstemmed', 'stemmed', 'score']

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv('maes_features.csv')

Can just use the features and score for the X and y but just to keep to certain convention if reading back from the CSV file above.


In [ ]:
X = dataset.iloc[:,0:13].values.astype(float)
y = dataset.iloc[:,14].values.astype(float)

In [ ]:
y

In [ ]:
X.shape

In [ ]:
y = np.array(y).reshape(-1,1)
y.shape

#### Conduct Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [ ]:
len(X)

In [ ]:
len(y)

#### Split the train and test sets

In [ ]:
# To split the train / test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Have a look at the first few lines
print(y_test[:5, :])

### Training

#### Support Vector Regression

In [ ]:
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# maybe use poly and increase the degree
regressor = SVR(kernel='rbf', gamma='auto', verbose=True)
#regressor = SVR(kernel='poly', degree=5, gamma='auto', verbose=True)
regressor.fit(X_train,y_train.ravel())

#### Test / Predict the fit

In [ ]:
# Not used yet as I don't have a sample X
y_pred = regressor.predict(X_test)
y_pred = sc_y.inverse_transform(y_pred).round()

In [ ]:
df = pd.DataFrame(
    {
        'Real Values':sc_y.inverse_transform(y_test.reshape(-1)),
        'Predicted Values':y_pred
    }
)
df.head()

#### Accuracy Score

In [ ]:
# y_pred

In [ ]:
# y_test = sc_y.inverse_transform(y_test).round()
# y_test.ravel()

In [ ]:
# Need to wrap my head around this (where's the predictor)

print("accuracy score:", regressor.score(X_test, y_test))

In [ ]:
print("accuracy score:", accuracy_score(df['Real Values'], df['Predicted Values']))

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
print(cohen_kappa_score(sc_y.inverse_transform(y_test).round(), y_pred, weights="quadratic"))

### Naive Bayes

In [ ]:
X_train

In [ ]:
X_train_test = sc_X.inverse_transform(X_train)
X_train_test = X_train_test.astype(int)
X_train_test

In [ ]:
y_train_test = sc_y.inverse_transform(y_train.reshape(-1))
y_train_test = y_train_test.astype(int)
y_train_test

In [ ]:
nbclassifier = naive_bayes.MultinomialNB()
nbclassifier.fit(X_train_test, y_train_test)

In [ ]:
X_test_test = sc_X.inverse_transform(X_test)
X_test_test = X_test_test.astype(int)
X_test_test

In [ ]:
y_test_test = sc_y.inverse_transform(y_test.reshape(-1))
y_test_test = y_test_test.astype(int)
y_test_test

In [ ]:
y_predNB = nbclassifier.predict(X_test_test)

cm = confusion_matrix(y_test_test, y_predNB)
print(cm)

In [ ]:
from sklearn.metrics import classification_report

rpt = classification_report(y_test_test, y_predNB)
print(rpt)

### QWK Scores (Manual Code)

In [ ]:
N = len(cm) # Just to get the same size as the confusion matrix from above
w = np.zeros((N,N)) # create a matrix of N by N
d = (N-1)**2 # the weighted portion
for i in range(len(w)):
    for j in range(len(w)):
        w[i][j] = float(((i-j)**2)/d) 
w # The weighted matrix

In [ ]:
N

In [ ]:
np.unique(y_test_test)

In [ ]:
np.unique(y_predNB)

In [ ]:
act_hist=np.zeros([N])
for item in y_test_test: 
    act_hist[item-1] += 1

In [ ]:
pred_hist=np.zeros([N])
for item in y_predNB: 
    pred_hist[item-1]+=1

In [ ]:
E = np.outer(act_hist, pred_hist)
E

In [ ]:
E = E/E.sum()
E.sum()

In [ ]:
cm = cm/cm.sum()
cm.sum()

In [ ]:
num=0
den=0
for i in range(len(w)):
    for j in range(len(w)):
        num+=w[i][j]*cm[i][j]
        den+=w[i][j]*E[i][j]
            
weighted_kappa = (1 - (num/den))
weighted_kappa

QWK scores output are from -1 to 1, where -1 means that it is totally wrong while 1 is a perfect match (classification).  The aim is to get as close as possible to 1, with a score of 0.6 being generally accepted as a good score.

### QWK for Naive Bayes

The above code is a manual computation of the QWK, which we later found that it is already available as an option with the [Cohen Kappa Score](https://journals.sagepub.com/doi/10.1177/001316446002000104) in sklearn, when we set the weights to 'quadratic'.  Since it has already been manually coded above, we use the sklearn.metrics.cohen_kappa_score to validate our manual coded scoring. 

In [ ]:
y_test_test

In [ ]:
y_predNB

In [ ]:
print(cohen_kappa_score(y_test_test, y_predNB))
print(cohen_kappa_score(y_test_test, y_predNB, weights="quadratic"))

On the output of the QWK agreements, the score is just "moderate agreement".  Work now is to achieve substantial agreement.

https://www.statisticshowto.com/cohens-kappa-statistic/

In short, SVM works a little better than Naive Bayes for AES.

# End